# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

In [10]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks

In [50]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Get one-year price returns for each stock in the universe

In [51]:
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

## Executing A Batch API Call & Building Our DataFrame

In [52]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [53]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)

## Removing Low-Momentum Stocks

In [55]:
#Sort the data by one year price return and modif the dataframe 
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
#final_dataframe

## Calculating the Number of Shares to Buy

In [56]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:12
12


In [57]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.92,2.30159,0
1,DVN,62.90,1.99501,0
2,CF,114.63,1.44001,0
3,MRO,26.26,1.39227,0
4,MOS,72.33,1.37251,0
5,APA,42.80,1.26378,0
6,NVDA,288.75,1.18342,0
7,OXY,60.79,1.13736,0
8,NUE,162.66,1.04651,0
9,MCHP,78.27,1.03445,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [58]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
        
hqm_dataframe
#sample {'stats': {'companyName': 'Apple Inc', 'marketcap': 2906649695548, 'week52high': 183.22, 'week52low': 118.58, 'week52highSplitAdjustOnly': 184.24, 'week52lowSplitAdjustOnly': 123.81, 'week52change': 0.4590356708388876, 'sharesOutstanding': 17071721882, 'float': 0, 'avg10Volume': 98736612, 'avg30Volume': 96835231, 'day200MovingAvg': 164.85, 'day50MovingAvg': 170.21, 'employees': 151847, 'ttmEPS': 6.06, 'ttmDividendRate': 0.8918965530362816, 'dividendYield': 0.005052822602038006, 'nextDividendDate': '', 'exDividendDate': '2022-01-21', 'nextEarningsDate': '2022-04-18', 'peRatio': 28.832598430998722, 'beta': 1.274280817610036, 'maxChangePercent': 66.36819313804286, 'year5ChangePercent': 4.419820354551062, 'year2ChangePercent': 1.7926784140277439, 'year1ChangePercent': 0.4610852807959017, 'ytdChangePercent': -0.015046682317266252, 'month6ChangePercent': 0.2020579818678947, 'month3ChangePercent': -0.007927948415889978, 'month1ChangePercent': 0.06099540974120701, 'day30ChangePercent': 0.07645137075533028, 'day5ChangePercent': 0.06587804333471159}, 'quote': {'avgTotalVolume': 96192991, 'calculationPrice': 'close', 'change': 0.68, 'changePercent': 0.0038, 'close': 178.35, 'closeSource': 'cilifofa', 'closeTime': 1725711981878, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 175.32, 'delayedPriceTime': 1651469998628, 'extendedChange': 0.24, 'extendedChangePercent': 0.00137, 'extendedPrice': 183.19, 'extendedPriceTime': 1722230813144, 'high': 179.59, 'highSource': 'nmeuid5ye1 eaildp t ecr', 'highTime': 1690132591717, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 178.36, 'iexCloseTime': 1710810165609, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 180.64, 'iexOpenTime': 1668711010258, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1693833434094, 'latestPrice': 179.93, 'latestSource': 'Close', 'latestTime': 'March 25, 2022', 'latestUpdate': 1670157982780, 'latestVolume': 82744953, 'low': 177.11, 'lowSource': 'ytilem r5a p1ecd eeidun', 'lowTime': 1713392949985, 'marketCap': 2916118727481, 'oddLotDelayedPrice': 175.34, 'oddLotDelayedPriceTime': 1712896048038, 'open': 177.11, 'openTime': 1729920717866, 'openSource': 'flaicfoi', 'peRatio': 29.99, 'previousClose': 177.44, 'previousVolume': 92970865, 'primaryExchange': 'NAQADS', 'symbol': 'AAPL', 'volume': 81010697, 'week52High': 190.67, 'week52Low': 123.41, 'ytdChange': -0.011098844201449035, 'isUSMarketOpen': False}}


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.80,N/A,0.0865224,N/A,-0.219855,N/A,-0.144617,N/A,0.0151027,N/A,N/A
1,AAL,17.06,N/A,-0.260531,N/A,-0.206464,N/A,-0.0689778,N/A,-0.0269029,N/A,N/A
2,AAP,217.62,N/A,0.18108,N/A,0.011684,N/A,-0.0674487,N/A,0.0580049,N/A,N/A
3,AAPL,182.44,N/A,0.467449,N/A,0.192708,N/A,-0.00784494,N/A,0.0600196,N/A,N/A
4,ABBV,163.27,N/A,0.597658,N/A,0.566127,N/A,0.227923,N/A,0.0819834,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.34,N/A,0.131594,N/A,-0.0402331,N/A,-0.116565,N/A,-0.0337026,N/A,N/A
498,ZBH,128.14,N/A,-0.228852,N/A,-0.17105,N/A,-0.013779,N/A,-0.0115559,N/A,N/A
499,ZBRA,443.44,N/A,-0.111819,N/A,-0.250651,N/A,-0.27033,N/A,0.0181808,N/A,N/A
500,ZION,74.03,N/A,0.312469,N/A,0.193888,N/A,0.145798,N/A,0.00126942,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [59]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,98.19,199,0.444090,0.885149,0.147456,0.837624,0.221461,0.473267,0.093820,0.530693,0.681683
1,AAL,13.89,1411,-0.526494,0.0237624,-0.564540,0.0158416,0.509431,0.908911,0.172430,0.827723,0.444059
2,AAP,161.13,121,0.088066,0.578218,0.148378,0.839604,0.295700,0.643564,0.144608,0.742574,0.70099
3,AAPL,467.65,41,1.171724,0.992079,0.401695,0.968317,0.474900,0.887129,0.189840,0.879208,0.931683
4,ABBV,97.29,201,0.478770,0.89703,0.001711,0.629703,0.077880,0.19802,-0.024533,0.0693069,0.448515
5,ABC,104.97,186,0.163705,0.651485,0.100112,0.774257,0.241867,0.522772,0.066990,0.364356,0.578218
6,ABMD,305.78,64,0.546138,0.942574,0.874783,0.99802,0.645795,0.954455,0.145153,0.746535,0.910396
7,ABT,102.94,190,0.161073,0.647525,0.139130,0.823762,0.093917,0.233663,0.081815,0.457426,0.540594
8,ACN,237.88,82,0.197484,0.69505,0.085519,0.762376,0.279570,0.605941,0.067045,0.366337,0.607426
9,ADBE,453.91,43,0.532839,0.934653,0.196705,0.879208,0.252382,0.550495,0.006829,0.150495,0.628713


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()